In [ ]:
import requests
import os 
from pathlib import Path
import pandas as pd, pyarrow as pa
import glob 

# Tarefa 1: Setup e Consolidação dos Dados
1. Download automatizado:

    * Códigos para baixar arquivos mensais de 2023 e 2024 (meta: 24 meses, mínimo: 12
        meses)

    * Encontrar os dados e o padrão da URL em:
        `https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page`

    * Focar apenas nos Yellow Taxi Trip Records

    * Verificar integridade dos arquivos e tratar erros

In [ ]:
# Padrão:
# ano em yyyy
# mes em mm
# `https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{ano}-{mes}.parquet`

def baixar_arquivo_com_pathlib(url, pasta_destino):
    pasta = Path(pasta_destino)
    # Garante que a pasta de destino exista
    pasta.mkdir(parents=True, exist_ok=True)

    nome_arquivo = url.split('/')[-1]
    caminho_local = pasta / nome_arquivo 

    print(f"Baixando '{nome_arquivo}' de '{url}'...")
    try:
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(caminho_local, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"Download concluído com sucesso! Arquivo salvo em: {caminho_local}")
        return caminho_local
    except requests.exceptions.RequestException as e:
        print(f"Erro ao baixar o arquivo: {e}")
        return None

url_do_arquivo = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
pasta_para_salvar = "data"

meses = [f"{i:02d}" for i in range(1, 13)]
anos = ['2023','2024']

# PODE QUEBRAR COM OS DADOS DE 2025 QUE NÃO TEM OS DADOS DE TODOS OS MESES MESES 
for ano in anos:
    for mes in meses:
        
        baixar_arquivo_com_pathlib(
            f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{ano}-{mes}.parquet", 
            pasta_para_salvar
        )

# VERIFICAR A INTEGRIDADE DOS ARQUIVOS 

#print(f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{ano}-{mes}.parquet")


## 2. Análise de schema:
* Examine estrutura dos arquivos Parquet
* Identifique e cuide de mudanças de schema entre diferentes meses

In [ ]:
#Verificando os arquivos

# Ler arquivo parquet

meses = [f"{i:02d}" for i in range(1, 13)]
anos = ['2023','2024']

base_path = "data"  # pasta onde estão os arquivos
schemas = {}

for ano in anos:
    for mes in meses:
        file_name = f"yellow_tripdata_{ano}-{mes}.parquet"
        file_path = os.path.join(base_path, file_name)

        if os.path.exists(file_path):
            try:
                # Lê apenas o schema sem carregar tudo na memória
                df = pd.read_parquet(file_path, engine="fastparquet")

                # Guarda colunas e tipos em dicionário
                schemas[file_name] = {col: str(dtype) for col, dtype in df.dtypes.items()}
            except Exception as e:
                print(f"Erro ao ler {file_name}: {e}")

# Agora vamos comparar os schemas
all_columns = set()
for schema in schemas.values():
    all_columns.update(schema.keys())

print("\n=== Diferenças detectadas ===\n")
for file, schema in schemas.items():
    missing = all_columns - set(schema.keys())
    extra = set(schema.keys()) - all_columns
    if missing or extra:
        print(f"{file}:")
        if missing:
            print(f"  - Faltando colunas: {missing}")
        if extra:
            print(f"  - Colunas extras: {extra}")

# Comparar tipos coluna a coluna
print("\n=== Divergências de tipos ===\n")
for col in all_columns:
    tipos = {}
    for file, schema in schemas.items():
        if col in schema:
            tipos[file] = schema[col]
    if len(set(tipos.values())) > 1:
        print(f"Coluna '{col}' com tipos diferentes:")
        for file, tipo in tipos.items():
            print(f"  {file}: {tipo}")

In [ ]:
# Correção destes problemas

files = glob.glob("data/yellow_tripdata_*.parquet")

# Colunas padrão (ajustadas para minúsculas)
colunas_padrao = [
    "vendorid", "tpep_pickup_datetime", "tpep_dropoff_datetime",
    "passenger_count", "trip_distance", "ratecodeid", "store_and_fwd_flag",
    "pulocationid", "dolocationid", "payment_type", "fare_amount",
    "extra", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge",
    "total_amount", "congestion_surcharge", "airport_fee"
]

# Tipos alvo
dtype_fix = {
    "vendorid": "Int32",
    "pulocationid": "Int32",
    "dolocationid": "Int32",
    "payment_type": "Int32",
    "passenger_count": "Int32",
    "trip_distance": "float64",
    "fare_amount": "float64",
    "total_amount": "float64",
    "tpep_pickup_datetime": "datetime64[ns]",
    "tpep_dropoff_datetime": "datetime64[ns]",
}

dfs = []

for f in files:
    df = pd.read_parquet(f, engine="pyarrow")
    
    # padronizar nomes
    df.columns = df.columns.str.lower()
    
    # garantir colunas padrão (faltantes = NaN)
    df = df.reindex(columns=colunas_padrao)
    
    # aplicar dtypes
    for col, tipo in dtype_fix.items():
        if col in df.columns:
            try:
                df[col] = df[col].astype(tipo)
            except Exception:
                # fallback: se não converter, deixa como está
                df[col] = pd.to_numeric(df[col], errors="coerce") if "float" in tipo or "int" in tipo else df[col]
    
    dfs.append(df)

# Concatenar
df_final = pd.concat(dfs, ignore_index=True)

# Salvar unificado
df_final.to_parquet("yellow_tripdata_all.parquet", engine="pyarrow", compression="snappy")


## 3. Limpeza de dados
* Remover registros com duração de viagem negativa ou zero
* Filtrar coordenadas inválidas (fora dos ranges de latitude/longitude)
* Eliminar valores negativos em campos monetários
* Remover registros com passenger_count = 0
* Validar datas dentro do período esperado